<a href="https://colab.research.google.com/github/chenchongsong/udacity-cs344-colab/blob/main/notebook/udacity_cs344_hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Homework 5 for Udacity CS344 Course, Intro to Parallel Programming
# clone the code repo,
!git clone https://github.com/chenchongsong/udacity-cs344-colab
!pip install git+git://github.com/depctg/nvcc4jupyter.git

# load cuda plugin
%config NVCCPluginV2.static_dir = True
%config NVCCPluginV2.relative_dir = "udacity-cs344-colab/src/HW5"
%load_ext nvcc_plugin

# change to work directory, generate makefiles
!mkdir udacity-cs344-colab/build
%cd udacity-cs344-colab/build
!cmake ../src

In [ ]:
%%cuda --name student.cu

/* Udacity HW5
   Histogramming for Speed

   The goal of this assignment is compute a histogram
   as fast as possible.  We have simplified the problem as much as
   possible to allow you to focus solely on the histogramming algorithm.

   The input values that you need to histogram are already the exact
   bins that need to be updated.  This is unlike in HW3 where you needed
   to compute the range of the data and then do:
   bin = (val - valMin) / valRange to determine the bin.

   Here the bin is just:
   bin = val

   so the serial histogram calculation looks like:
   for (i = 0; i < numElems; ++i)
     histo[val[i]]++;

   That's it!  Your job is to make it run as fast as possible!

   The values are normally distributed - you may take
   advantage of this fact in your implementation.

*/


#include "utils.h"
#include <thrust/device_vector.h>
#include <thrust/sort.h>

const int BLOCK_SIZE = 1024;  // must be larger than numBins

__global__
void yourHistoBaseline(
    const unsigned int* const d_vals, //INPUT
    unsigned int* const d_histo,      //OUTPUT
    const unsigned int numBins,
    const unsigned int numElems) {
  int myId = threadIdx.x + blockDim.x * blockIdx.x;
  int tid  = threadIdx.x;

  // create a private histogram copy for each thread block
  extern __shared__ unsigned int hist[];
  for (int i = tid; i < numBins; i += blockDim.x) {  // 基于HW3，但考虑得更周全
    hist[i] = 0;
  }
  __syncthreads();

  // update each block's private histogram
  if (myId < numElems) {
    atomicAdd(&(hist[d_vals[myId]]), 1);
  }
  __syncthreads();

  for (int i = tid; i < numBins; i += blockDim.x) {
    atomicAdd(&(d_histo[i]), hist[i]); 
  }
}

void computeHistogram(const unsigned int* const d_vals, //INPUT
                      unsigned int* const d_histo,      //OUTPUT
                      const unsigned int numBins,
                      const unsigned int numElems) {
  //numBins: 1024, numElems: 10240000

  //v1: baseline, 3.32ms
  //int numBlock = ceil(1.0f * numElems / BLOCK_SIZE);
  //int shBytes = numBins * sizeof(unsigned int);
	//yourHistoBaseline<<<numBlock, BLOCK_SIZE, shBytes>>>(d_vals, d_histo, numBins, numElems);

  
}

In [ ]:
# make the cuda project
!make HW5
print("\n====== RESULT OF HW5 =======\n")
!bin/HW5